## Combine Rating Datasets

1. Encode qualitative ratings with quantitative values
2. Normalize brand names
3. Deal with duplicates
4. Join dataframes on brand name

In [ ]:
# imports
import pandas as pd

In [ ]:
# read in datasets
eco_stylist = pd.read_csv("../data/eco_stylist_ratings.csv")
sustainable_review = pd.read_csv("../data/sustainable_review_ratings.csv")
good_on_you_recommended = pd.read_csv("../data/gou_recommended.csv")
good_on_you = pd.read_csv("../data/brand_info.csv")

eco_stylist.head(5)

,Unnamed: 0,Brand,Overall,Transparency,Fair Labor,Sustainably Made,URL
0,0,KNOWN SUPPLY,Certified,Good,Excellent,Good,https://www.eco-stylist.com/ethical-brand/know...
1,1,AMENDI,Certified,Excellent,Fair,Good,https://www.eco-stylist.com/ethical-brand/amendi/
2,2,SANVT,Certified,Excellent,Good,Good,https://www.eco-stylist.com/ethical-brand/sanvt/
3,3,Patagonia,Gold,Excellent,Excellent,Excellent,https://www.eco-stylist.com/ethical-brand/pata...
4,4,Scotch & Soda,Silver,Excellent,Fair,Excellent,https://www.eco-stylist.com/ethical-brand/scot...


In [ ]:
# correct index column for sustainable_review and eco_stylist
eco_stylist.drop(columns='Unnamed: 0', inplace=True)
sustainable_review.drop(columns='Unnamed: 0', inplace=True)

eco_stylist.head(5)

,Brand,Overall,Transparency,Fair Labor,Sustainably Made,URL
0,KNOWN SUPPLY,Certified,Good,Excellent,Good,https://www.eco-stylist.com/ethical-brand/know...
1,AMENDI,Certified,Excellent,Fair,Good,https://www.eco-stylist.com/ethical-brand/amendi/
2,SANVT,Certified,Excellent,Good,Good,https://www.eco-stylist.com/ethical-brand/sanvt/
3,Patagonia,Gold,Excellent,Excellent,Excellent,https://www.eco-stylist.com/ethical-brand/pata...
4,Scotch & Soda,Silver,Excellent,Fair,Excellent,https://www.eco-stylist.com/ethical-brand/scot...


In [ ]:
# check column types for eco-stylist prior to encoding
eco_stylist.dtypes

Brand               object
Overall             object
Transparency        object
Fair Labor          object
Sustainably Made    object
URL                 object
dtype: object

In [ ]:
# order categorical rankings in rating columns
eco_stylist['Overall'] = pd.Categorical(eco_stylist['Overall'], categories=['Certified', 'Silver', 'Gold'], ordered=True)
subratings = ['Transparency', 'Fair Labor', 'Sustainably Made']
for subrating in subratings:
    eco_stylist[subrating] = pd.Categorical(eco_stylist[subrating], categories=['Good', 'Fair', 'Excellent'], ordered=True)

eco_stylist.dtypes

Brand                 object
Overall             category
Transparency        category
Fair Labor          category
Sustainably Made    category
URL                   object
dtype: object

In [ ]:
# encode categorical rankings from eco_stylist into quantiative ratings
eco_stylist_encoded = eco_stylist.copy()
categorical_columns = eco_stylist_encoded.select_dtypes(['category']).columns
eco_stylist_encoded[categorical_columns] = eco_stylist_encoded[categorical_columns].apply(lambda x: x.cat.codes + 1)

eco_stylist_encoded.head(5)

,Brand,Overall,Transparency,Fair Labor,Sustainably Made,URL
0,KNOWN SUPPLY,1,1,3,1,https://www.eco-stylist.com/ethical-brand/know...
1,AMENDI,1,3,2,1,https://www.eco-stylist.com/ethical-brand/amendi/
2,SANVT,1,3,1,1,https://www.eco-stylist.com/ethical-brand/sanvt/
3,Patagonia,3,3,3,3,https://www.eco-stylist.com/ethical-brand/pata...
4,Scotch & Soda,2,3,2,3,https://www.eco-stylist.com/ethical-brand/scot...


In [ ]:
# scale quantiative ratings up from 1-3 to 1-5
eco_stylist_scaled = eco_stylist_encoded.copy()
rating_columns = eco_stylist_scaled.select_dtypes(['int8']).columns
eco_stylist_scaled[rating_columns] = eco_stylist_scaled[rating_columns].apply(lambda x: round(((5/3) * x), 1))

eco_stylist_scaled.head(5)

,Brand,Overall,Transparency,Fair Labor,Sustainably Made,URL
0,KNOWN SUPPLY,1.7,1.7,5.0,1.7,https://www.eco-stylist.com/ethical-brand/know...
1,AMENDI,1.7,5.0,3.3,1.7,https://www.eco-stylist.com/ethical-brand/amendi/
2,SANVT,1.7,5.0,1.7,1.7,https://www.eco-stylist.com/ethical-brand/sanvt/
3,Patagonia,5.0,5.0,5.0,5.0,https://www.eco-stylist.com/ethical-brand/pata...
4,Scotch & Soda,3.3,5.0,3.3,5.0,https://www.eco-stylist.com/ethical-brand/scot...


In [ ]:
# normalize brand names function
def normalize_brand_names(df, brand_col):
    try:
        norm_brands = []
        for brand in df[brand_col]: 
            # drop all non-special characters
            norm_brands.append(''.join(e.lower() for e in brand if e.isalnum()))
        
        # create new column for normalized brand names
        df['norm_brands'] = norm_brands

    except Exception as e:
        print(e)

In [ ]:
# normalize eco-stylist brand names
normalize_brand_names(eco_stylist_scaled, 'Brand')

# check that all brand names remain unique
print("# of brand names: " + str(len(eco_stylist_scaled['Brand'].unique())))
print("# of normalized brand names: " + str(len(eco_stylist_scaled['norm_brands'].unique())))

eco_stylist_scaled.head(5)

# of brand names: 102
# of normalized brand names: 102


,Brand,Overall,Transparency,Fair Labor,Sustainably Made,URL,norm_brands
0,KNOWN SUPPLY,1.7,1.7,5.0,1.7,https://www.eco-stylist.com/ethical-brand/know...,knownsupply
1,AMENDI,1.7,5.0,3.3,1.7,https://www.eco-stylist.com/ethical-brand/amendi/,amendi
2,SANVT,1.7,5.0,1.7,1.7,https://www.eco-stylist.com/ethical-brand/sanvt/,sanvt
3,Patagonia,5.0,5.0,5.0,5.0,https://www.eco-stylist.com/ethical-brand/pata...,patagonia
4,Scotch & Soda,3.3,5.0,3.3,5.0,https://www.eco-stylist.com/ethical-brand/scot...,scotchsoda


In [ ]:
# normalize sustainable review brand names and check that all brand names remain unique
normalize_brand_names(sustainable_review, 'Brand')

# check that all brand names remain unique
print("# of brand names: " + str(len(sustainable_review['Brand'].unique())))
print("# of normalized brand names: " + str(len(sustainable_review['norm_brands'].unique())))

sustainable_review.head(5)

# of brand names: 1140
# of normalized brand names: 1140


,Brand,Rating,Description,URL,norm_brands
0,AARVEN,4,The AARVEN brand is a renowned apparel and jew...,https://sustainablereview.com/brand-ratings/aa...,aarven
1,A A K S,4,A A K S is a unique and sustainable brand that...,https://sustainablereview.com/brand-ratings/a-...,aaks
2,Aestethic London,5,Aestethic London is an environmentally conscio...,https://sustainablereview.com/brand-ratings/ae...,aestethiclondon
3,A-dam,4,A-dam is a sustainable fashion brand that is m...,https://sustainablereview.com/brand-ratings/a-...,adam
4,Adele Dejak,3,Adele Dejak is a brand that is making efforts ...,https://sustainablereview.com/brand-ratings/ad...,adeledejak


In [ ]:
# normalize good on you brand names and check that all brand names remain unique
normalize_brand_names(good_on_you, 'brand')

# check that all brand names remain unique
print("# of brand names: " + str(len(good_on_you['brand'].unique())))
print("# of normalized brand names: " + str(len(good_on_you['norm_brands'].unique())))

good_on_you.head(5)

# of brand names: 105
# of normalized brand names: 105


,brand,overall_rating,planet_score,people_score,animals_score,description,norm_brands
0,Princess Polly,2,2.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,princesspolly
1,Brandy Melville,1,1.0,1.0,0.0,This brand provides insufficient relevant info...,brandymelville
2,Shein,1,1.0,1.0,2.0,Our “Planet” rating evaluates brands based on ...,shein
3,Nike,3,3.0,3.0,2.0,Our “Planet” rating evaluates brands based on ...,nike
4,Abercrombie & Fitch,2,2.0,2.0,2.0,Abercrombie & Fitch is owned by Abercrombie Ab...,abercrombiefitch


In [ ]:
# normalize recommendedgood on you brand names and check that all brand names remain unique
normalize_brand_names(good_on_you_recommended, 'brand')

# check that all brand names remain unique
print("# of brand names: " + str(len(good_on_you_recommended['brand'].unique())))
print("# of normalized brand names: " + str(len(good_on_you_recommended['norm_brands'].unique())))

good_on_you_recommended.head(5)

# of brand names: 454
# of normalized brand names: 453


,brand,overall_rating,planet_score,people_score,animals_score,description,norm_brands
0,Enfant Terrible,4,5,4,4.0,Enfant Terrible's environment rating is 'great...,enfantterrible
1,milo+nicki,4,5,3,4.0,milo+nicki's environment rating is 'great'. It...,milonicki
2,Birdsong,4,5,5,3.0,Birdsong's environment rating is 'great'. It u...,birdsong
3,DAYWEARLAB,4,5,3,3.0,DAYWEARLAB's environment rating is 'great'. It...,daywearlab
4,LOVETRUST,4,4,3,5.0,LOVETRUST's environment rating is 'good'. It u...,lovetrust


In [ ]:
# identify duplicate normalized brand name 
norm_brands = []
for brand in good_on_you_recommended['norm_brands']:
    if brand not in norm_brands:
        # add brand to list if it is not a duplicate
        norm_brands.append(brand)
    else:
        print("Duplicate brand name: " + brand)

Duplicate brand name: mori


In [ ]:
# extract columns with duplicate normalized brand name
good_on_you_recommended[good_on_you_recommended['norm_brands'] == 'mori']

,brand,overall_rating,planet_score,people_score,animals_score,description,norm_brands
73,MORI,3,3,2,4.0,MORI's environment rating is 'it's a start'. I...,mori
124,mori,4,5,4,4.0,Our “Planet” rating evaluates brands based on ...,mori


In [ ]:
# clean and rename columns of all datasets
eco_stylist_scaled.drop(columns='URL', inplace=True)
eco_stylist_scaled.rename(columns={'Brand': 'es_brand', 'Overall':'es_rating', 'Transparency':'es_transparency', 'Fair Labor':'es_fair_labor', 'Sustainably Made':'es_sustainably_made'}, inplace=True)

sustainable_review.drop(columns='URL', inplace=True)
sustainable_review.rename(columns={'Brand': 'sr_brand', 'Rating':'sr_rating', 'Description':'sr_description'}, inplace=True)

good_on_you.rename(columns={'brand': 'goy_brand', 'overall_rating':'goy_rating', 'planet_score':'goy_planet', 'people_score':'goy_people', 'animals_score':'goy_animals', 'description':'goy_description'}, inplace=True)

good_on_you_recommended.rename(columns={'brand': 'goyr_brand', 'overall_rating':'goyr_rating', 'planet_score':'goyr_planet', 'people_score':'goyr_people', 'animals_score':'goyr_animals', 'description':'goyr_description'}, inplace=True)

In [ ]:
# check for duplicate normalized brand name in other datasets
for df in [eco_stylist_scaled, sustainable_review, good_on_you]:
    for brand in df['norm_brands']:
        if 'mori' == brand:
            duplicate = df[df['norm_brands'] == 'mori']

            print(duplicate)
            print(duplicate.index)

    sr_brand  sr_rating                                     sr_description  \
652     mori          4  Our “Planet” rating evaluates brands based on ...   

    norm_brands  
652        mori  
Index([652], dtype='int64')


In [ ]:
# manually alter duplicate normalized brand name to join datasets
good_on_you_recommended['norm_brands'].loc[good_on_you_recommended['goyr_brand'] == 'MORI'] = 'mori2'
good_on_you_recommended[good_on_you_recommended['goyr_brand'] == 'MORI']

C:\Users\vfang\AppData\Local\Temp\ipykernel_49640\3074476147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_on_you_recommended['norm_brands'].loc[good_on_you_recommended['goyr_brand'] == 'MORI'] = 'mori2'


,goyr_brand,goyr_rating,goyr_planet,goyr_people,goyr_animals,goyr_description,norm_brands
73,MORI,3,3,2,4.0,MORI's environment rating is 'it's a start'. I...,mori2


In [ ]:
# join datasets on normalized brand names
two_brands = pd.merge(eco_stylist_scaled, sustainable_review, how='outer', on='norm_brands')
three_brands = pd.merge(two_brands, good_on_you, how='outer', on='norm_brands')
all_brand_ratings = pd.merge(three_brands, good_on_you_recommended, how='outer', on='norm_brands')

all_brand_ratings.head(5)

,es_brand,es_rating,es_transparency,es_fair_labor,es_sustainably_made,norm_brands,sr_brand,sr_rating,sr_description,goy_brand,...,goy_planet,goy_people,goy_animals,goy_description,goyr_brand,goyr_rating,goyr_planet,goyr_people,goyr_animals,goyr_description
0,KNOWN SUPPLY,1.7,1.7,5.0,1.7,knownsupply,Known Supply,4.0,"<a data-wpel-link=""external"" href=""https://kno...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMENDI,1.7,5.0,3.3,1.7,amendi,AMENDI,4.0,AMENDI is a brand that is making commendable e...,NaN,...,NaN,NaN,NaN,NaN,AMENDI,4.0,4.0,3.0,4.0,AMENDI's environment rating is 'good'. It uses...
2,SANVT,1.7,5.0,1.7,1.7,sanvt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Patagonia,5.0,5.0,5.0,5.0,patagonia,Patagonia,4.0,Patagonia is a brand that focuses on sustainab...,Patagonia,...,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,Patagonia,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...
4,Scotch & Soda,3.3,5.0,3.3,5.0,scotchsoda,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# compare lengths of individual datasets and joined datasets
print("Total length of individual datasets: " + str(len(eco_stylist_scaled) + len(sustainable_review) + len(good_on_you) + len(good_on_you_recommended)))
print("Total length of joined datasets: " + str(len(all_brand_ratings)))

NameError: name 'eco_stylist_scaled' is not defined

In [ ]:
# combine brand name columns from each dataset
all_brand_ratings['brand'] = all_brand_ratings['es_brand']
all_brand_ratings['brand'].loc[all_brand_ratings['brand'].isnull()] = all_brand_ratings['sr_brand']
all_brand_ratings['brand'].loc[all_brand_ratings['brand'].isnull()] = all_brand_ratings['goy_brand']
all_brand_ratings['brand'].loc[all_brand_ratings['brand'].isnull()] = all_brand_ratings['goyr_brand']

print("Contains empty brand names: " + str(all_brand_ratings['brand'].isnull().values.any()))

Contains empty brand names: False


C:\Users\vfang\AppData\Local\Temp\ipykernel_49640\3720915540.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_brand_ratings['brand'].loc[all_brand_ratings['brand'].isnull()] = all_brand_ratings['sr_brand']
C:\Users\vfang\AppData\Local\Temp\ipykernel_49640\3720915540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_brand_ratings['brand'].loc[all_brand_ratings['brand'].isnull()] = all_brand_ratings['goy_brand']
C:\Users\vfang\AppData\Local\Temp\ipykernel_49640\3720915540.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [ ]:
# drop redundant brand name columns
all_brand_ratings.drop(columns=['es_brand', 'sr_brand', 'goy_brand', 'goyr_brand', 'norm_brands'], inplace=True)
all_brand_ratings.head(5)

,es_rating,es_transparency,es_fair_labor,es_sustainably_made,sr_rating,sr_description,goy_rating,goy_planet,goy_people,goy_animals,goy_description,goyr_rating,goyr_planet,goyr_people,goyr_animals,goyr_description,brand
0,1.7,1.7,5.0,1.7,4.0,"<a data-wpel-link=""external"" href=""https://kno...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KNOWN SUPPLY
1,1.7,5.0,3.3,1.7,4.0,AMENDI is a brand that is making commendable e...,NaN,NaN,NaN,NaN,NaN,4.0,4.0,3.0,4.0,AMENDI's environment rating is 'good'. It uses...,AMENDI
2,1.7,5.0,1.7,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SANVT
3,5.0,5.0,5.0,5.0,4.0,Patagonia is a brand that focuses on sustainab...,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,Patagonia
4,3.3,5.0,3.3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Scotch & Soda


In [ ]:
# reorder columns
columns = list(all_brand_ratings.columns)
columns.insert(0, columns.pop(columns.index('brand')))
all_brand_ratings = all_brand_ratings.loc[:, columns]

all_brand_ratings.head(5)

,brand,es_rating,es_transparency,es_fair_labor,es_sustainably_made,sr_rating,sr_description,goy_rating,goy_planet,goy_people,goy_animals,goy_description,goyr_rating,goyr_planet,goyr_people,goyr_animals,goyr_description
0,KNOWN SUPPLY,1.7,1.7,5.0,1.7,4.0,"<a data-wpel-link=""external"" href=""https://kno...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMENDI,1.7,5.0,3.3,1.7,4.0,AMENDI is a brand that is making commendable e...,NaN,NaN,NaN,NaN,NaN,4.0,4.0,3.0,4.0,AMENDI's environment rating is 'good'. It uses...
2,SANVT,1.7,5.0,1.7,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Patagonia,5.0,5.0,5.0,5.0,4.0,Patagonia is a brand that focuses on sustainab...,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...
4,Scotch & Soda,3.3,5.0,3.3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# export csv file
all_brand_ratings.to_csv('../data/all_brand_ratings.csv')

#### Findings & Limitations
- The function .to_csv() sometimes creates an unnamed column based on the index
- Scaling up ratings for eco-stylist to match sustainable review and good on you creates float values